import & setup cơ bản

In [10]:
import pandas as pd
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)
import glob

input file statement quarter, concat, and export to clean data

In [11]:
path_quarterly = '../../data/raw/datavietstock/all_statement_hsx_hnx_upcom/quarterly/editted/'   # path to statement quarter
# thay đổi path ở dòng path và data_clean để lấy dữ liệu yearly hoặc quarterly
path = path_quarterly
# tạo danh sách đường dẫn fle
report_files = glob.glob(path + "*.xlsx")    # lấy dữ liệu ở path quaterly
list_report = []  # tạo list rỗng để append các BCTC
a = 0
for filename in report_files:
    a =+1
    read_BCTC = pd.read_excel(filename,sheet_name=0, header=None)
    #read_BCTC = read_BCTC.fillna(method='ffill')
    read_BCTC.columns = read_BCTC.iloc[0]   #lấy tên columns ở dòng đầu tiền
    BCTC = read_BCTC[1:]  # chỉ lấy dữ liệu ở dòng thứ 2 trở đi
    #BCTC.set_index('Ticker', inplace = True)
    list_report.append(BCTC)    # tạo list các báo cao doanh nghiệp

report_concat = pd.concat(list_report, axis = 0 )  # nối các báo cáo doanh nghiệp
report_concat_cleaned = report_concat.dropna(subset=['18. Net profit after tax  (60 = 50 - 51 - 52)']) # xóa các cp không có bctc

# Do du lieu vietstock ve CF chi co o dang cong don 6,9,12 thang nen de tinh quy phai tru q2-q1,q3-q2,q4-q3
start_col_name = '1. Profit before tax'
end_col_name = 'Close Price End of period'
# Lấy chỉ số của các cột bắt đầu và kết thúc
start_col_index = report_concat_cleaned.columns.get_loc(start_col_name)
end_col_index = report_concat_cleaned.columns.get_loc(end_col_name)

# Sử dụng slicing để chọn các cột cần tính
columns_to_calculate = report_concat_cleaned.columns[start_col_index:end_col_index]
# Tính số lượng tăng lên cho các cột đã chọn
for column in columns_to_calculate:
    report_concat_cleaned[f'{column} Increase'] = report_concat_cleaned.groupby(['Stock code', 'Year'])[column].diff().fillna(report_concat_cleaned[column])

# Gán các giá trị tăng lên vào chính các cột ban đầu
for column in columns_to_calculate:
    report_concat_cleaned[column] = report_concat_cleaned[f'{column} Increase']
    
# Xóa các cột Increase nếu không cần thiết
report_concat_cleaned.drop(columns=[f'{column} Increase' for column in columns_to_calculate], inplace=True)

#report_concat
#report_concat_cleaned


C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_1900\188510823.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_concat_cleaned[f'{column} Increase'] = report_concat_cleaned.groupby(['Stock code', 'Year'])[column].diff().fillna(report_concat_cleaned[column])
C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_1900\188510823.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_concat_cleaned[f'{column} Increase'] = report_concat_cleaned.groupby(['Stock code', 'Year'])[column].diff().fillna(report_co

Thêm phân nhóm ngành theo ICB của finpro, vì phân loại như icb dễ phân tích hơn

In [12]:
#path_ICB = '../../data/raw/datavietstock/all_statement_hsx_hnx_upcom/quarterly/Sector_ICB_finpro.xlsx'   # path to ICB fle
#ICB = pd.read_excel(path_ICB,sheet_name=0, header=0)
path_VS = '../../data/raw/datavietstock/all_statement_hsx_hnx_upcom/quarterly/Sector_VS.xlsx'   # path to ICB fle
VS = pd.read_excel(path_VS,sheet_name=0, header=0)
report_concat_cleaned = pd.merge(report_concat_cleaned,VS, on='Stock code', how='outer')
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
report_concat_cleaned.to_excel(path_bctc_clean_quarterly+"BCTC_quarterly.xlsx", index=False)
report_concat_cleaned.to_csv(path_bctc_clean_quarterly+"BCTC_quarterly.csv", index=False)
report_concat_cleaned

,No.,Stock code,Company name,Exchange,Full Name Vietnamese,Sector (NAICS) Level 1,Sector (NAICS) Level 2,Sector (NAICS) Level 3,Year,Quarter,Year_Q,A. SHORT-TERM ASSETS (100=110+120+130+140+150),I. Cash and cash equivalents,1. Cash,2. Cash equivalents,II. Short-term financial investments,1. Available for sale securities,2. Provision for diminution in value of available for sale securities (*),3. Held to maturity investments,III. Short-term receivables,1. Short-term trade accounts receivable,2. Short-term prepayments to suppliers,3. Short-term inter-company receivables,4. Construction contract progress receipts due from customers,5. Short-term loan receivables,6. Other short-term receivables,7. Provision for short-term doubtful debts (*),8. Assets awaiting resolution,IV. Inventories,1. Inventories,2. Provision for decline in value of inventories,V. Other short-term assets,1. Short-term prepayments,2. Value added tax to be reclaimed,3. Taxes and other receivables from state authorities,4. Government bonds,5. Other short-term assets,B. LONG-TERM ASSETS (200=210+220+240+250+260+270),I. Long-term receivables,1. Long-term trade receivables,2. Long-term prepayments to suppliers,3. Capital at inter-company,4. Long-term inter-company receivables,5. Long-term loan receivables,6. Other long-term receivables,7. Provision for long-term doubtful debts,II. Fixed assets,1. Tangible fixed assets,- Cost,- Accumulated depreciation,2. Financial leased fixed assets,- Cost,- Accumulated depreciation,3. Intangible fixed assets,- Cost,- Accumulated depreciation,III. Investment properties,- Cost,- Accumulated depreciation,IV. Long-term assets in progress,1. Long-term production in progress,2. Construction in progress,V. Long-term financial investments,1. Investments in subsidiaries,"2. Investments in associates, joint-ventures",3. Investments in other entities,4. Provision for diminution in value of long-term investments,5. Held to maturity investments,6. Other long-term investments,VI. Other long-term assets,1. Long-term prepayments,2. Deferred income tax assets,"3. Long-term equipment, supplies, spare parts",4. Other long-term assets,VII. Goodwill,TOTAL ASSETS (280=100+200),A. LIABILITIES (300=210+330),I. Short -term liabilities,1. Short-term trade accounts payable,2. Short-term advances from customers,3. Taxes and other payables to state authorities,4. Payable to employees,5. Short-term acrrued expenses,6. Short-term inter-company payables,7. Construction contract progress payments due to suppliers,8. Short-term unearned revenue,9. Other short-term payables,10. Short-term borrowings and financial leases,11. Provision for short-term liabilities,12.. Bonus and welfare fund,13. Price stabilization fund,14. Government bonds,II. Long-term liabilities,1. Long-term trade payables,2. Long-term advances from customers,3. Long-term acrrued expenses,4. Inter-company payables on business capital,5. Long-term inter-company payables,6. Long-term unearned revenue,7. Other long-term liabilities,8. Long-term borrowings and financial leases,9. Convertible bonds,10. Preferred stock (Debts),11. Deferred income tax liabilities,12. Provision for long-term liabilities,13. Fund for technology development,14. Provision for severance allowances,B. OWNER'S EQUITY (400=410+430),I. Owner's equity,1. Owner's capital,- Common stock with voting right,- Preferred stock,2. Share premium,3. Convertible bond option,4. Other capital of owners,5. Treasury shares,6. Assets revaluation differences,7. Foreign exchange differences,8. Investment and development fund,9. Fund to support corporate restructuring,10. Other funds from owner's equity,11. Undistributed earnings after tax,- Accumulated retained earning at the end of the previous period,- Undistributed earnings in this period,12. Reserves for investment in construction,13. Minority's interest,14. Financial reserves,II. Other resources and funds,1. Subsidized not-for-profit funds,2. Funds invested in fixed assets,C. MINORITY'S INTERES

Report toàn ngành loại bđs, bank, airline

In [13]:
Sector_level1_exclude = ['Financials','Real Estate']
Sector_level4_exclude = ['Passenger Airlines ']

BCTC_none_financials_realestate = report_concat_cleaned[~report_concat_cleaned['VS-Sector L1'].isin(Sector_level1_exclude)] 
BCTC_none_financials_realestate_airline = BCTC_none_financials_realestate[~BCTC_none_financials_realestate['VS-Sector L4'].isin(Sector_level4_exclude)] 
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
BCTC_none_financials_realestate_airline.to_excel(path_bctc_clean_quarterly+"BCTC_none_financials_realestate_airline.xlsx", index=False)
BCTC_none_financials_realestate_airline.to_csv(path_bctc_clean_quarterly+"BCTC_none_financials_realestate_airline.csv", index=False)

#